In [1]:
import os 
import datetime

In [2]:
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
plt.style.use('seaborn') 
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

In [4]:
data = pd.read_excel('sale_data_v03_0820.xlsx', encoding = 'utf-8')  
sale = pd.read_excel('sale_meta_v03_0820.xlsx', encoding = "utf-8")
item = pd.read_excel('item_meta_v02_0820.xlsx', encoding = "utf-8")

# Data 

## sale_data_v03_0820.xlsx
* 2019실적데이터 + NEW상품명 

In [5]:
data.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,단위,브랜드,옵션,종류
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,866.0,테이트 여성 셀린니트 3종,0.0,3종,테이트,NaN,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,866.0,테이트 여성 셀린니트 3종,0.0,3종,테이트,NaN,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38309 entries, 0 to 38308
Data columns (total 15 columns):
방송일시       38309 non-null datetime64[ns]
노출(분)      21525 non-null float64
마더코드       38309 non-null int64
상품코드       38309 non-null int64
상품명        38309 non-null object
상품군        38309 non-null object
판매단가       38309 non-null int64
취급액        35379 non-null float64
NEW상품코드    37372 non-null float64
NEW상품명     37372 non-null object
결제방법       37372 non-null float64
단위         22390 non-null object
브랜드        33066 non-null object
옵션         4140 non-null object
종류         22744 non-null object
dtypes: datetime64[ns](1), float64(4), int64(3), object(7)
memory usage: 4.4+ MB


In [7]:
data["상품군"].unique() # '무형' 있는 데이터

array(['의류', '속옷', '주방', '농수축', '이미용', '가전', '생활용품', '건강기능', '잡화', '무형',
       '가구', '침구'], dtype=object)

# 하나의 데이터로 묶기..!
* '테이트 남성 셀린니트3종' 상품은 2019-01-01 06:00 ~ 2019-01-01 07:00 까지 방송했지만, 행이 따로 따로 구성되어 있음을 알 수 있음 
* 이를 하나로 합쳐주는 작업이 필요함.....! 

In [8]:
data.loc[data["상품명"] == "테이트 남성 셀린니트3종"]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,단위,브랜드,옵션,종류
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0
164,2019-01-03 00:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,7329000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0
166,2019-01-03 00:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,10481000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0


In [9]:
# 일단 하나의 방송 프로그램만 보기 = 노출 NaN 제외하고 보기 
# 방송 프로그램 내의 상품 순서는..과연 동일한가 ...... ^_^ ?? 

bct = data.dropna(subset = ["노출(분)"])
bct.shape

(21525, 15)

In [10]:
print(bct["상품명"].nunique(), bct["NEW상품명"].nunique()) # NEW상품명 기준으로 맞춰 주기로 함 
bct.loc[bct["상품군"] == "무형"]["상품명"].nunique() # 937+78 = 1015 이므로 대략적으로 안 맞는 값은 '무형'에서 발생, 나머지는 체크해 주어야 함 

1076 937


78

In [11]:
# 무형 데이터 동일 상품명으로 채워주기 
bct["NEW상품명"] = bct["NEW상품명"].fillna(bct["상품명"])

In [12]:
bct["date"] = bct["방송일시"].dt.date

In [13]:
bct.loc[bct["NEW상품명"] == "멋진밥상 흥양농협 햅쌀 20kg"].head(5)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,단위,브랜드,옵션,종류,date
34,2019-01-01 16:40:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,농수축,61000,83893000.0,1241.0,멋진밥상 흥양농협 햅쌀 20kg,0.0,20kg,NaN,NaN,쌀,2019-01-01
385,2019-01-05 16:00:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,농수축,61000,44297000.0,1241.0,멋진밥상 흥양농협 햅쌀 20kg,0.0,20kg,NaN,NaN,쌀,2019-01-05
386,2019-01-05 16:20:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,농수축,61000,65082000.0,1241.0,멋진밥상 흥양농협 햅쌀 20kg,0.0,20kg,NaN,NaN,쌀,2019-01-05
387,2019-01-05 16:40:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,농수축,61000,69522000.0,1241.0,멋진밥상 흥양농협 햅쌀 20kg,0.0,20kg,NaN,NaN,쌀,2019-01-05
578,2019-01-07 15:00:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,농수축,61000,31814000.0,1241.0,멋진밥상 흥양농협 햅쌀 20kg,0.0,20kg,NaN,NaN,쌀,2019-01-07


In [14]:
bct.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,단위,브랜드,옵션,종류,date
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0,2019-01-01
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0,2019-01-01
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0,2019-01-01
6,2019-01-01 07:00:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,6819000.0,632.0,오모떼 여성 레이스 파운데이션 브라,0.0,NaN,오모떼,NaN,4,2019-01-01
7,2019-01-01 07:20:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,15689000.0,632.0,오모떼 여성 레이스 파운데이션 브라,0.0,NaN,오모떼,NaN,4,2019-01-01


In [16]:
bct.shape

(21525, 16)

In [17]:
dup = bct.copy()
dup["중복"] = bct.duplicated(["date", "NEW상품명"])
dup = dup.loc[dup["중복"] == True]
dup.shape

(14308, 17)

In [19]:
dup.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,단위,브랜드,옵션,종류,date,중복
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0,2019-01-01,True
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0,2019-01-01,True
7,2019-01-01 07:20:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,15689000.0,632.0,오모떼 여성 레이스 파운데이션 브라,0.0,NaN,오모떼,NaN,4,2019-01-01,True
8,2019-01-01 07:40:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,25370000.0,632.0,오모떼 여성 레이스 파운데이션 브라,0.0,NaN,오모떼,NaN,4,2019-01-01,True
10,2019-01-01 08:20:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,30061000.0,1079.0,CERINI by PAT 남성 소프트 기모 릴렉스 팬츠,0.0,NaN,CERINI by PAT,NaN,1,2019-01-01,True


In [22]:
dup.groupby(["NEW상품명"]).agg("unique" : "nunique")

SyntaxError: invalid syntax (<ipython-input-22-e85e2f3342ea>, line 1)

In [21]:
dup = dup.loc[dup.groupby(["NEW상품명"]).agg("unique" : "nunique") >= 2]
dup.shape

SyntaxError: invalid syntax (<ipython-input-21-0ac5747dbb8a>, line 1)

In [37]:
# 상품명만 가지고 다 drop하면 방송을 여러 번 한 상품들도 다 없어지게 됨 
# 방송 날짜 & 상품명 겹치는 것으로 체크 
# 이렇게 되면 11:40 ~ 00:40 방송한 상품은... 남게 됨 ..! (이건 어떻게 해아 할까 ㅠㅠ) -> 없는것보단 나으므로 나중에 생각하기 
# 동일 날짜에 여러번 방송한 상품들은 날라가게 됨 ^_^ ... ... ... 

time = bct.drop_duplicates(["date", 'NEW상품명'], keep='first')

In [48]:
time.loc[time["NEW상품명"] == "멋진밥상 흥양농협 햅쌀 20kg"].head(5)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,단위,브랜드,옵션,종류,date
34,2019-01-01 16:40:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,농수축,61000,83893000.0,1241.0,멋진밥상 흥양농협 햅쌀 20kg,0.0,20kg,NaN,NaN,쌀,2019-01-01
385,2019-01-05 16:00:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,농수축,61000,44297000.0,1241.0,멋진밥상 흥양농협 햅쌀 20kg,0.0,20kg,NaN,NaN,쌀,2019-01-05
578,2019-01-07 15:00:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,농수축,61000,31814000.0,1241.0,멋진밥상 흥양농협 햅쌀 20kg,0.0,20kg,NaN,NaN,쌀,2019-01-07
653,2019-01-08 14:00:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,농수축,61000,36664000.0,1241.0,멋진밥상 흥양농협 햅쌀 20kg,0.0,20kg,NaN,NaN,쌀,2019-01-08
772,2019-01-09 17:00:00,20.0,100849,202512,멋진밥상 흥양농협 햅쌀 20kg,농수축,61000,45915000.0,1241.0,멋진밥상 흥양농협 햅쌀 20kg,0.0,20kg,NaN,NaN,쌀,2019-01-09


In [49]:
time.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,단위,브랜드,옵션,종류,date
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0,2019-01-01
6,2019-01-01 07:00:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,6819000.0,632.0,오모떼 여성 레이스 파운데이션 브라,0.0,NaN,오모떼,NaN,4,2019-01-01
9,2019-01-01 08:00:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,16133000.0,1079.0,CERINI by PAT 남성 소프트 기모 릴렉스 팬츠,0.0,NaN,CERINI by PAT,NaN,1,2019-01-01
12,2019-01-01 09:00:00,20.0,100816,202404,보코 리버시블 무스탕,의류,79000,16075000.0,378.0,보코 리버시블 무스탕,0.0,NaN,보코,NaN,2,2019-01-01
15,2019-01-01 10:00:00,20.0,100809,202395,CERINI by PAT 남성 풀패키지 기모니트 3종,의류,79900,48799000.0,1089.0,CERINI by PAT 남성 풀패키지 기모니트 3종,0.0,3종,CERINI by PAT,NaN,0,2019-01-01


In [51]:
time["다음방송"] = time["방송일시"].shift(-1)

C:\Users\LG\venv\jbeen\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
time["방송시간"] = time["다음방송"] - time["방송일시"]

C:\Users\LG\venv\jbeen\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
time.head(25)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,단위,브랜드,옵션,종류,date,다음방송,방송시간
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,865.0,테이트 남성 셀린니트 3종,0.0,3종,테이트,NaN,0,2019-01-01,2019-01-01 07:00:00,01:00:00
6,2019-01-01 07:00:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,6819000.0,632.0,오모떼 여성 레이스 파운데이션 브라,0.0,NaN,오모떼,NaN,4,2019-01-01,2019-01-01 08:00:00,01:00:00
9,2019-01-01 08:00:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,16133000.0,1079.0,CERINI by PAT 남성 소프트 기모 릴렉스 팬츠,0.0,NaN,CERINI by PAT,NaN,1,2019-01-01,2019-01-01 09:00:00,01:00:00
12,2019-01-01 09:00:00,20.0,100816,202404,보코 리버시블 무스탕,의류,79000,16075000.0,378.0,보코 리버시블 무스탕,0.0,NaN,보코,NaN,2,2019-01-01,2019-01-01 10:00:00,01:00:00
15,2019-01-01 10:00:00,20.0,100809,202395,CERINI by PAT 남성 풀패키지 기모니트 3종,의류,79900,48799000.0,1089.0,CERINI by PAT 남성 풀패키지 기모니트 3종,0.0,3종,CERINI by PAT,NaN,0,2019-01-01,2019-01-01 11:00:00,01:00:00
18,2019-01-01 11:00:00,20.0,100664,201999,크로커다일 The 편안한 코튼 브라팬티 4세트,속옷,69900,24531000.0,831.0,크로커다일 여성 The 편안한 코튼 브라팬티 4세트,0.0,4세트,크로커다일,NaN,4,2019-01-01,2019-01-01 12:00:00,01:00:00
20,2019-01-01 12:00:00,20.0,100088,200236,에코라믹 통주물 스톤 냄비세트,주방,60900,33823000.0,587.0,에코라믹 통주물 스톤 냄비세트,0.0,NaN,에코라믹,NaN,NaN,2019-01-01,2019-01-01 13:00:00,01:00:00
23,2019-01-01 13:00:00,30.0,100305,200970,오모떼 360도 텐션업 레이스 패키지 시즌4,속옷,109000,32273000.0,629.0,오모떼 여성 360도 텐션업 레이스 패키지,0.0,NaN,오모떼,시즌4,4,2019-01-01,2019-01-01 14:00:00,01:00:00
26,2019-01-01 14:00:00,30.0,100377,201220,그렉노먼 남성 구스다운 롱 벤치코트,의류,119000,11269000.0,22.0,그렉노먼 남성 구스다운 롱 벤치코트,0.0,NaN,그렉노먼,NaN,2,2019-01-01,2019-01-01 15:00:00,01:00:00
29,2019-01-01 15:00:00,20.0,100496,201497,뽕셰프 이봉원 특갈비탕 12팩,농수축,60900,14113000.0,407.0,뽕셰프 이봉원 특갈비탕 12팩,0.0,12팩,뽕셰프,NaN,육류,2019-01-01,2019-01-01 16:00:00,01:00:00


In [68]:
time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7217 entries, 0 to 38307
Data columns (total 18 columns):
방송일시       7217 non-null datetime64[ns]
노출(분)      7217 non-null float64
마더코드       7217 non-null int64
상품코드       7217 non-null int64
상품명        7217 non-null object
상품군        7217 non-null object
판매단가       7217 non-null int64
취급액        6794 non-null float64
NEW상품코드    6919 non-null float64
NEW상품명     7217 non-null object
결제방법       6919 non-null float64
단위         3630 non-null object
브랜드        5573 non-null object
옵션         904 non-null object
종류         4507 non-null object
date       7217 non-null object
다음방송       7216 non-null datetime64[ns]
방송시간       7216 non-null timedelta64[ns]
dtypes: datetime64[ns](2), float64(4), int64(3), object(8), timedelta64[ns](1)
memory usage: 1.4+ MB


In [74]:
# check 1 : 방송시간이 긴 경우 (다음 날짜로 넘어가는 경우) -> 제대로 채워주는 작업 필요 
# check 2 : 

time.loc[time["방송시간"] > datetime.timedelta(hours=1)]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,단위,브랜드,옵션,종류,date,다음방송,방송시간
64,2019-01-02 01:00:00,20.0,100193,200652,레이프릴 무빙 맥시풀커버 브라팬티,속옷,69900,9647000.0,140.0,레이프릴 여성 무빙 맥시풀커버 브라팬티,0.0,NaN,레이프릴,NaN,4,2019-01-02,2019-01-02 06:00:00,05:00:00
168,2019-01-03 01:00:00,20.0,100613,201894,푸마 웜셀 기모 치마레깅스 3종,속옷,59000,13185000.0,928.0,푸마 여성 웜셀 기모 치마레깅스 3종,0.0,3종,푸마,NaN,4,2019-01-03,2019-01-03 06:00:00,05:00:00
263,2019-01-04 01:00:00,20.0,100229,200800,벨라홈 논슬립 옷걸이 세트,생활용품,29800,9780000.0,322.0,벨라홈 논슬립 옷걸이 세트,0.0,NaN,벨라홈,NaN,NaN,2019-01-04,2019-01-04 06:00:00,05:00:00
345,2019-01-05 01:00:00,20.0,100293,200952,일시불[가이거] 제니스시계 주얼리세트,잡화,129000,15067000.0,4.0,가이거 제니스시계 주얼리세트,1.0,NaN,가이거,NaN,주얼리,2019-01-05,2019-01-05 06:00:00,05:00:00
388,2019-01-05 17:00:00,20.0,100435,201350,우리바다 손질왕꼬막 24팩,농수축,39900,52050000.0,1297.0,우리바다 손질왕꼬막 24팩,0.0,24팩,NaN,NaN,해산물,2019-01-05,2019-01-05 18:20:00,01:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38041,2019-12-28 22:20:00,20.0,100317,201004,국내산 손질 햇 갑오징어 9팩,농수축,49900,52262000.0,1210.0,국내산 손질 햇 갑오징어 9팩,0.0,9팩,NaN,NaN,해산물,2019-12-28,2019-12-29 00:00:00,01:40:00
38050,2019-12-29 01:20:00,20.0,100723,202114,아이앤아이 퀵래쉬 속눈썹세트,이미용,59000,16544000.0,543.0,아이앤아이 퀵래쉬 속눈썹세트,0.0,NaN,아이앤아이,NaN,NaN,2019-12-29,2019-12-29 06:20:00,05:00:00
38087,2019-12-29 17:20:00,20.0,100010,200014,AAB의 소곱창전골 800g x 8팩,농수축,55900,41093000.0,1360.0,AAB의 소곱창 전골 800g 8팩,0.0,"800g, 8팩",NaN,NaN,간편식품,2019-12-29,2019-12-29 20:20:00,03:00:00
38135,2019-12-30 01:20:00,20.0,100226,201021,코이모 리빙박스 8종,생활용품,50800,10217000.0,780.0,코이모 리빙박스 8종,0.0,8종,코이모,NaN,NaN,2019-12-30,2019-12-30 06:20:00,05:00:00


In [80]:
time.loc[time["상품명"] == "온라인투어 북경 (191227) 상담예약"] # 수작업 각인디 ... 

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,단위,브랜드,옵션,종류,date,다음방송,방송시간
37941,2019-12-27 23:20:00,20.0,100324,201098,온라인투어 북경 (191227) 상담예약,무형,0,NaN,NaN,온라인투어 북경 (191227) 상담예약,NaN,NaN,NaN,NaN,NaN,2019-12-27,2019-12-28 00:00:00,00:40:00
37943,2019-12-28 00:00:00,20.0,100324,201098,온라인투어 북경 (191227) 상담예약,무형,0,NaN,NaN,온라인투어 북경 (191227) 상담예약,NaN,NaN,NaN,NaN,NaN,2019-12-28,2019-12-28 00:20:00,00:20:00
38046,2019-12-29 00:00:00,20.0,100324,201098,온라인투어 북경 (191227) 상담예약,무형,0,NaN,NaN,온라인투어 북경 (191227) 상담예약,NaN,NaN,NaN,NaN,NaN,2019-12-29,2019-12-29 00:20:00,00:20:00
